In [2]:
import rdflib
from rdflib.namespace import RDF


g = rdflib.Graph()
g.parse("Graph - FateGO.omn")
print(list(g)[:10])

FGO = rdflib.Namespace("http://kg@se.ifmo.ru/~s367583/fgo#")
list(g.objects(FGO["Target_ptAll"], RDF.type))

for prefix, uri in g.namespaces():
    print(f"Prefix: {prefix}, URI: {uri}")

[(rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#Skill_692350'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividual')), (rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#faterequiem-collab-challenge-guide-child-underworld-utsumi-erice'), rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#hasRecommendedDamageServant'), rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#Servant_Unknown')), (rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#Buff_2649'), rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#targetsTrait'), rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#Trait_5000')), (rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#Skill_199550/Effect_0'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividual')), (rdflib.term.URIRef('http://kg@se.ifmo.ru/~s367583/fgo#Skill_157350/Effect_0'

In [3]:
# list all Servants that can Buff Quick for the party for at least 10 %
query = """
PREFIX : <http://kg@se.ifmo.ru/~s367583/fgo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?servant ?skill ?value
WHERE {
    ?svt rdfs:label ?servant ;
             :hasSkill ?sk .
    ?sk rdfs:label ?skill ;
            :hasEffect ?e .
        
    ?e :hasFunction ?f ;
       :hasValue ?v .
    ?f :grantsBuff ?b ;
       :targets :Target_ptAll .
    ?b :hasBuffType :BuffType_upCommandall ;
      :requiresSelfTrait :Trait_4003 .
    ?v :value ?value .
    FILTER(?value >= 10)
}
"""
for row in g.query(query, ):
    print(f"{str(row.servant)!r} by {row.value}% via skill {str(row.skill)!r}")
print("done")

'Helena Blavatsky' by 20% via skill 'Search for the Unknown B'
'Taigong Wang' by 15% via skill 'Primitive Tactics A+'
'Zhang Jue' by 20% via skill 'General of Heaven B-'
'Scáthach-Skadi (Ruler)' by 15% via skill 'Midsummer Frozen Treat C'
'Sen-no-Rikyu' by 20% via skill 'Finest Wabi A-'
'Alexander' by 20% via skill 'Omen of the Conqueror A'
'Atalante' by 36% via skill 'Beyond Arcadia B'
'Atalante' by 50% via skill 'Beyond Arcadia A'
'Atalante (Alter)' by 50% via skill 'Beyond Arcadia A'
'Altria Pendragon Alter (Rider)' by 20% via skill 'Summer Sweeper! A'
'Ishtar (Rider)' by 20% via skill 'Shining Water Robe A'
'Odysseus' by 20% via skill "Resourceful General's Inspiration B++"
done


In [4]:
# List all Servants and Skills that grant advantage against Roman Trait
query = """
PREFIX : <http://kg@se.ifmo.ru/~s367583/fgo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?servant ?skill ?value
WHERE {
    ?svt rdfs:label ?servant ;
             :hasSkill ?sk .
    ?sk rdfs:label ?skill .

    ?sk :hasEffect ?e1 .
    ?e1 :hasFunction ?f1.
    ?f1 :targetsTrait :Trait_2004.
}
"""
for row in g.query(query, ):
    print(f"{str(row.servant)!r} via skill {str(row.skill)!r}")
print("done")

'Kōnstantînos XI' via skill "Empire's Decline EX"
'Romulus=Quirinus' via skill 'Throne of Quirinus EX'
done


In [7]:
query = """
PREFIX : <http://kg@se.ifmo.ru/~s367583/fgo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?Atk ?Def ?Skill ?Trait
WHERE {
    ?svt1 rdfs:label ?Atk.
    ?svt2 rdfs:label ?Def.

    ?svt1 :hasTrait ?c1.
    ?c1 rdf:type :Class.
    ?svt2 :hasTrait ?c2.
    ?c2 rdf:type :Class.

    ?c1 :effectiveAgainst ?c2.

    ?svt1 :hasSkill ?s.
    ?s rdfs:label ?Skill.
    
    ?s :hasEffect ?e.
    ?e :hasFunction ?f.
    ?f :grantsBuff ?b.
    ?b :requiresEnemyTrait ?t.
    ?svt2 :hasTrait ?t.
    ?t rdfs:label ?Trait.

    ?b :hasBuffTrait :BuffTrait_upCommandall.
}
"""
for row in g.query(query, ):
    print(f"{str(row.Atk)!r} vs {str(row.Def)!r} via {str(row.Skill)!r} (trait {str(row.Trait)!r})")